In [1]:
import matplotlib.pyplot as plt
import warnings
import numpy as np
import pandas as pd
import os
import cv2
import json
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torchvision.io import read_image



warnings.filterwarnings('ignore')

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, names=['file_name', 'label'], skiprows=[0])
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        try:
            image = read_image(img_path)
        except:
            print(self.img_labels.iloc[idx, 0])
            exit()
        label = int(self.img_labels.iloc[idx, 1])
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [3]:
#Define Neural Networks Model.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 10)

    def forward(self, x):
        x = x.float()
        h1 = F.relu(self.fc1(x.view(-1, 784)))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        h4 = F.relu(self.fc4(h3))
        h5 = F.relu(self.fc5(h4))
        h6 = self.fc6(h5)
        return F.log_softmax(h6, dim=1)

In [4]:
#Prepare Data Loader for Training and Validation

transform = transforms.Compose([
                 transforms.ToTensor(),
                 transforms.Normalize((0.1307,), (0.3081,))])

print("init model done")

init model done


In [5]:
epochs = 10
lr = 0.01
momentum = 0.5
no_cuda = True
seed = 1
log_interval = 200

use_cuda = not no_cuda and torch.cuda.is_available()

torch.manual_seed(seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

print("set vars and device done")

set vars and device done


In [6]:
batch_size = 64
test_batch_size = 1000
dataset = CustomImageDataset(
    annotations_file='./data/FashionMNIST/annotations.csv',
    img_dir='./data/FashionMNIST/imgs',
    )

In [13]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

# Change the dataset in test_loader to other scv file to check the result
test_loader = torch.utils.data.DataLoader(dataset,
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [8]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [9]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [10]:
def test(log_interval, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format
          (test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
for epoch in range(1, 11):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(log_interval, model, device, test_loader)
torch.save(model, './model.pt')

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.372018
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.378668
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.365031
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.303907
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.851829

Test set: Average loss: 2.5930, Accuracy: 6000/60000 (10%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.709135
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.774677
Train Epoch: 2 [25600/60000 (43%)]	Loss: 3.235073
Train Epoch: 2 [38400/60000 (64%)]	Loss: 3.487649
Train Epoch: 2 [51200/60000 (85%)]	Loss: 4.158109

Test set: Average loss: 5.0738, Accuracy: 6000/60000 (10%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 5.362549
Train Epoch: 3 [12800/60000 (21%)]	Loss: 4.753544
Train Epoch: 3 [25600/60000 (43%)]	Loss: 6.357253
Train Epoch: 3 [38400/60000 (64%)]	Loss: 5.970634
Train Epoch: 3 [51200/60000 (85%)]	Loss: 5.366640

Test set: Average loss: 10.8377, Accuracy: 6000/60000 (10%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 8.729541
Train Epoch: 4 [12800/60000 (21%)]	L

In [12]:
with open(json_file, 'r') as f:
    annotations_file = json.load(f)

NameError: name 'json_file' is not defined